<a href="https://colab.research.google.com/github/byui-cse/cse480-notebooks/blob/master/11_2_Ponder_and_Prove_NP_Completeness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ponder and Prove NP-Completeness
## Due: Saturday, 20 March 2021, 11:59 pm

## TODO Watch and Learn

Watch each of the following videos:

* [P vs. NP - The Biggest Unsolved Problem in Computer Science](https://www.youtube.com/watch?v=EHp4FPyajKQ)
* [What is complexity theory? (P vs. NP explained visually)](https://www.youtube.com/watch?v=u2DLlNQiPB4)
* [P vs. NP and the Computational Complexity Zoo](https://www.youtube.com/watch?v=YX40hbAHx3s&t=3s)

For each video, state two things that interested you (theoretical or practical). Just a couple of sentences that summarize each point of interest.

## TODO Exercises

Do the following Chapter 16 Exercises:

### 16.7.2.1

A SAT instance is given in the DIMACS format.

(a) What is the CNF formula captured by this instance?

(b) By inspection, answer whether the instance is satisfiable, and why.

(c) If it is not satisfiable, then what is the minimal number of rows that must be deleted before the instance becomes satisfiable? If these rows are not unique, list the first two possible such omissions (of sets of rows), starting from the top of the given listing.

(d) Check your answer using CryptoMiniSat.

(a): (a+b+c+d+e).(a+b+c+d+!e).(a+b+c+!d+e).(a+b+c+!d+!e).(a+b+!c+d+e).(a+b+!c+d+!e).(a+b+!c+!d+e).(a+b+!c+!d+!e).(a+!b+c+d+e).(a+!b+c+d+!e).(a+!b+c+!d+e).(a+!b+c+!d+!e).(a+!b+!c+d+e).(a+!b+!c+d+!e).(a+!b+!c+!d+e).(a+!b+!c+!d+!e).(!a+b+c+d+e).(!a+b+c+d+!e).(!a+b+c+!d+e).(!a+b+c+!d+!e).(!a+b+!c+d+e).(!a+b+!c+d+!e).(!a+b+!c+!d+e).(!a+b+!c+!d+!e).(!a+!b+c+d+e).(!a+!b+c+d+!e).(!a+!b+c+!d+e).(!a+!b+c+!d+!e).(!a+!b+!c+d+e).(!a+!b+!c+d+!e).(!a+!b+!c+!d+e).(!a+!b+!c+!d+!e)
Thank you Kyle, but know that I do know how to write this by hand even if it would suck.

(b) It is not satisfiable, since at least one clause needs to be removed for it to be satisfied. Thank you Robert for helping me out here. It's been a rough day and night.

(c) Any row can be removed.

(d) 

### 16.7.2.2

Consider the set of undirected graphs $<G>$ with a set of nodes $N$ and a set of edges $E \subseteq N x N$ such that we can two-color the graph (meaning no two nodes connected by an edge  have the same color).

We can use a breadth-first search to color the graph by choosing a starting color for the first node then make every node connected to it the opposite color. Then go to those nodes and make every node connected to them the original color. From there switch colors back and forth but continue to branch out to uncolored nodes making them the opposite of what they are adjacent to. -Thanks Robert

### 16.7.2.8

A Hamiltonian cycle in a graph with respect to a given node $n$ is a tour that begins at $n$ and visits all other nodes exactly once, returning to $n$. In a clique, how many distinct Hamiltonian cycles exist? How about in an $n$-clique?

There are 5! distinct Hamiltonian cucles in a 5-clique graph. No labels, 4! in a 5-clique graph. $n!$ for $n$-clique for labeled ones, and $(n - 1)!$ for no lables.

### 16.7.2.9

Define the language $HalfClique$ to be the set of input encodings $<G>$ such that $G$ is an undirected graph having a clique with at least $n/2$ nodes, where $n$ is the number of nodes in $G$. Show that $HalfClique$ is NPC. $\textit{Hint: Mapping reduction from Clique.}$

Your answer goes here.

### 16.7.2.10

The game of Sudoku has been shown to be in NPC. In practice,  one can encode and solve Sudoku using SAT solvers. This is also a  good way to understand the power of modern SAT solvers. Study  the Sudoku solver (MIT license) written by Nicholas Pilkington at  https://gist.github.com/nickponline/9c91fe65fef5b58ae1b0.  Test it on the instance provided as well as a few that you create. Note: This solver will need Python2 (or you may adapt it for  Python3).

In [11]:
# Author: Nicholas Pilkington, 2015
# License: MIT
# Blog Post: https://nickp.svbtle.com/sudoku-satsolver

import pycosat

N = 9
M = 3

def exactly_one(variables):
    cnf = [ variables ]
    n = len(variables)

    for i in range(n):
        for j in range(i+1, n):
            v1 = variables[i]
            v2 = variables[j]
            cnf.append([-v1, -v2])

    return cnf

def transform(i, j, k):
    return i*N*N + j*N + k + 1

def inverse_transform(v):
    v, k = divmod(v-1, N)
    v, j = divmod(v, N)
    v, i = divmod(v, N)
    return i, j, k

if __name__ == '__main__':
    cnf = []

    # Cell, row and column constraints
    for i in range(N):
        for s in range(N):
            cnf = cnf + exactly_one([ transform(i, j, s) for j in range(N) ])
            cnf = cnf + exactly_one([ transform(j, i, s) for j in range(N) ])

        for j in range(N):
            cnf = cnf + exactly_one([ transform(i, j, k) for k in range(N) ])

    # Sub-matrix constraints
    for k in range(N):
        for x in range(M):
            for y in range(M):
                v = [ transform(y*M + i, x*M + j, k) for i in range(M) for j in range(M)]
                cnf = cnf + exactly_one(v)

    # See contribution from @GregoryMorse below
    cnf = { frozenset(x) for x in cnf }
    cnf = list(cnf)

    # A 16-constraint Sudoku
    constraints = [
        (0, 3, 8),
        (1, 0, 1),
        (2, 3, 4),
        (2, 4, 3),
        (2, 6, 2),
        (3, 8, 6),
        (4, 3, 5),
        (4, 5, 9),
        (5, 6, 4),
        (5, 7, 1),
        (5, 8, 8),
        (6, 4, 8),
        (6, 5, 1),
        (7, 2, 2),
        (7, 7, 5),
        (8, 1, 4),
        (8, 6, 3),

    ]

    cnf = cnf + [[transform(z[0], z[1], z[2])-1] for z in constraints]

    for solution in pycosat.itersolve(cnf):
        X = [ inverse_transform(v) for v in solution if v > 0]
        for i, cell in enumerate(sorted(X, key=lambda h: h[0] * N*N + h[1] * N)):
            print(cell[2]+1, end=" ")
            if (i+1) % N == 0: print("")
        print("")

7 6 4 8 1 2 9 3 5 
1 2 3 9 7 5 8 6 4 
9 5 8 4 3 6 2 7 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 9 5 7 6 3 4 1 8 
5 7 9 3 8 1 6 4 2 
3 8 2 6 9 4 1 5 7 
6 4 1 2 5 7 3 8 9 

7 6 4 8 1 2 9 3 5 
1 2 3 9 7 5 8 6 4 
8 5 9 4 3 6 2 7 1 
2 3 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
9 7 5 3 8 1 6 4 2 
3 8 2 6 9 4 1 5 7 
6 4 1 2 5 7 3 8 9 

7 6 4 8 1 2 9 3 5 
1 2 3 9 7 5 8 6 4 
8 5 9 4 3 6 2 7 1 
2 3 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
3 7 5 2 8 1 6 4 9 
6 8 2 3 9 4 1 5 7 
9 4 1 6 5 7 3 8 2 

7 6 4 8 1 2 9 3 5 
1 2 3 9 7 5 8 6 4 
8 9 5 4 3 6 2 7 1 
2 3 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
9 5 6 7 2 3 4 1 8 
5 7 9 3 8 1 6 4 2 
3 8 2 6 9 4 1 5 7 
6 4 1 2 5 7 3 8 9 

7 6 4 8 1 2 9 3 5 
1 2 3 9 7 5 8 6 4 
5 9 8 4 3 6 2 7 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 5 9 7 6 3 4 1 8 
9 7 5 3 8 1 6 4 2 
3 8 2 6 9 4 1 5 7 
6 4 1 2 5 7 3 8 9 

7 6 4 8 1 2 9 3 5 
1 2 3 9 7 5 8 6 4 
5 9 8 4 3 6 2 7 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 5 9 7 6 3 4 1 8 
3 7 5 2 8 1 6 4 9 
6 8 2 3

4 1 8 5 6 9 7 3 2 
5 9 6 7 2 3 4 1 8 
3 7 5 9 8 1 6 2 4 
8 6 2 3 7 4 1 5 9 
9 4 1 2 5 6 3 8 7 

3 5 7 8 1 2 9 6 4 
1 2 4 6 9 7 8 3 5 
8 6 9 4 3 5 2 7 1 
2 7 3 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
7 3 5 9 8 1 6 4 2 
6 8 2 3 7 4 1 5 9 
9 4 1 2 5 6 3 8 7 

3 5 7 8 1 2 9 6 4 
1 2 4 6 9 7 8 3 5 
6 8 9 4 3 5 2 7 1 
2 7 3 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
7 3 5 9 8 1 6 4 2 
8 6 2 3 7 4 1 5 9 
9 4 1 2 5 6 3 8 7 

3 6 7 8 1 2 9 4 5 
1 2 4 6 9 5 8 7 3 
8 5 9 4 3 7 2 6 1 
2 7 3 1 4 8 5 9 6 
4 1 8 5 6 9 7 3 2 
5 9 6 7 2 3 4 1 8 
7 3 5 9 8 1 6 2 4 
6 8 2 3 7 4 1 5 9 
9 4 1 2 5 6 3 8 7 

3 6 4 8 1 2 9 7 5 
1 2 7 6 9 5 8 4 3 
8 5 9 4 3 7 2 6 1 
2 7 3 1 4 8 5 9 6 
4 1 8 5 6 9 7 3 2 
5 9 6 7 2 3 4 1 8 
7 3 5 9 8 1 6 2 4 
6 8 2 3 7 4 1 5 9 
9 4 1 2 5 6 3 8 7 

3 6 4 8 1 2 9 7 5 
1 2 7 6 9 5 8 3 4 
8 5 9 4 3 7 2 6 1 
2 7 3 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
7 3 5 9 8 1 6 4 2 
6 8 2 3 7 4 1 5 9 
9 4 1 2 5 6 3 8 7 

3 2 4 8 1 7 9 6 5 
1 5 7 6 9 2 8 3 4 
8 6 9 

6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

4 3 7 8 1 2 9 6 5 
1 2 5 6 9 7 8 3 4 
9 6 8 4 3 5 2 7 1 
2 7 3 1 4 8 5 9 6 
8 1 4 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
3 5 9 2 8 1 6 4 7 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

4 6 5 8 1 2 9 3 7 
1 2 3 6 9 7 8 4 5 
9 7 8 4 3 5 2 6 1 
2 3 7 1 4 8 5 9 6 
8 1 4 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
3 5 9 2 8 1 6 7 4 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

4 2 3 8 1 7 9 6 5 
1 7 5 6 9 2 8 3 4 
9 6 8 4 3 5 2 7 1 
2 3 7 1 4 8 5 9 6 
8 1 4 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
3 5 9 2 8 1 6 4 7 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

7 3 4 8 1 2 9 6 5 
1 2 5 6 9 7 8 3 4 
8 6 9 4 3 5 2 7 1 
2 7 3 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
3 5 7 9 8 1 6 4 2 
6 8 2 3 7 4 1 5 9 
9 4 1 2 5 6 3 8 7 

7 3 4 8 1 2 9 6 5 
1 2 5 6 9 7 8 3 4 
6 8 9 4 3 5 2 7 1 
2 7 3 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
3 5 7 9 8 1 6 4 2 
8 6 2 3 7 4 1 5 9 
9 4 1 2 5 6 3 8 7 

7 3 4 8 1 2 9 6 5 
1 2 5 6 9 7 8 4 3 
6 8 9 4 3 5 2 7 1 
2 7 3 1 4 8 5 9 6 
4 1 8 5 6 9 7 3 2 
5 9 6 

3 7 5 8 1 2 9 6 4 
1 2 4 6 9 7 8 3 5 
6 8 9 4 3 5 2 7 1 
2 3 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
9 5 3 2 8 1 6 4 7 
8 6 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

2 3 5 8 1 7 9 6 4 
1 7 4 6 9 2 8 3 5 
6 8 9 4 3 5 2 7 1 
3 2 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
9 5 3 2 8 1 6 4 7 
8 6 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

2 3 5 8 1 7 9 6 4 
1 7 4 6 9 2 8 3 5 
8 6 9 4 3 5 2 7 1 
3 2 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
9 5 3 2 8 1 6 4 7 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

2 6 5 8 1 7 9 3 4 
1 3 4 6 9 2 8 7 5 
8 7 9 4 3 5 2 6 1 
3 2 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
9 5 3 2 8 1 6 4 7 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

2 6 4 8 1 7 9 3 5 
1 3 5 6 9 2 8 7 4 
8 7 9 4 3 5 2 6 1 
3 2 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
9 5 3 2 8 1 6 4 7 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

2 6 4 8 1 7 9 3 5 
1 3 5 6 9 2 8 4 7 
8 7 9 4 3 5 2 6 1 
3 2 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 7 2 3 4 1 8 
9 5 3 2 8 1 6 7 4 
6 8 2 3

9 6 5 7 2 3 4 1 8 
3 5 9 2 8 1 6 7 4 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

4 2 6 8 1 5 9 3 7 
1 7 3 6 9 2 8 4 5 
5 9 8 4 3 7 2 6 1 
2 3 7 1 4 8 5 9 6 
8 1 4 5 6 9 7 2 3 
9 6 5 7 2 3 4 1 8 
3 5 9 2 8 1 6 7 4 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

4 7 6 8 1 2 9 3 5 
1 2 3 6 9 5 8 4 7 
5 9 8 4 3 7 2 6 1 
2 3 7 1 4 8 5 9 6 
8 1 4 5 6 9 7 2 3 
9 6 5 7 2 3 4 1 8 
3 5 9 2 8 1 6 7 4 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

4 7 6 8 1 2 9 3 5 
1 2 3 6 9 5 8 7 4 
5 9 8 4 3 7 2 6 1 
2 3 7 1 4 8 5 9 6 
8 1 4 5 6 9 7 2 3 
9 6 5 7 2 3 4 1 8 
3 5 9 2 8 1 6 4 7 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

4 3 6 8 1 2 9 7 5 
1 2 7 6 9 5 8 3 4 
5 9 8 4 3 7 2 6 1 
2 7 3 1 4 8 5 9 6 
8 1 4 5 6 9 7 2 3 
9 6 5 7 2 3 4 1 8 
3 5 9 2 8 1 6 4 7 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

3 5 6 8 1 2 9 7 4 
1 2 4 6 9 7 8 3 5 
8 9 7 4 3 5 2 6 1 
2 7 3 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
9 6 5 7 2 3 4 1 8 
5 3 9 2 8 1 6 4 7 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

3 5 6 8 1 2 9 4 7 
1 2 4 6 9 7 8 3 5 
8 9 7 4 3 5 2 6 1 
2 7 3 

...

3 2 5 8 1 7 9 6 4 
1 7 4 9 6 2 8 3 5 
9 8 6 4 3 5 2 7 1 
4 3 7 1 2 8 5 9 6 
6 1 8 5 4 9 7 2 3 
2 5 9 6 7 3 4 1 8 
5 9 3 7 8 1 6 4 2 
8 6 2 3 9 4 1 5 7 
7 4 1 2 5 6 3 8 9 

3 2 4 8 1 7 9 6 5 
1 7 5 9 6 2 8 4 3 
9 8 6 4 3 5 2 7 1 
4 3 7 1 2 8 5 9 6 
6 1 8 5 4 9 7 3 2 
2 5 9 6 7 3 4 1 8 
5 9 3 7 8 1 6 2 4 
8 6 2 3 9 4 1 5 7 
7 4 1 2 5 6 3 8 9 

3 2 4 8 1 7 9 6 5 
1 7 5 9 6 2 8 3 4 
9 8 6 4 3 5 2 7 1 
4 3 7 1 2 8 5 9 6 
6 1 8 5 4 9 7 2 3 
2 5 9 6 7 3 4 1 8 
5 9 3 7 8 1 6 4 2 
8 6 2 3 9 4 1 5 7 
7 4 1 2 5 6 3 8 9 

3 2 4 8 1 7 9 6 5 
1 7 6 9 5 2 8 3 4 
9 8 5 4 3 6 2 7 1 
4 3 7 1 2 8 5 9 6 
6 1 8 5 4 9 7 2 3 
2 5 9 7 6 3 4 1 8 
5 9 3 2 8 1 6 4 7 
8 6 2 3 7 4 1 5 9 
7 4 1 6 9 5 3 8 2 

3 2 4 8 1 7 9 6 5 
1 7 5 9 6 2 8 3 4 
9 6 8 4 3 5 2 7 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 5 9 6 7 3 4 1 8 
5 9 3 7 8 1 6 4 2 
6 8 2 3 9 4 1 5 7 
7 4 1 2 5 6 3 8 9 

3 2 4 8 1 7 9 6 5 
1 7 5 9 6 2 8 4 3 
9 6 8 4 3 5 2 7 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 3 2 
2 5 9 6 7 3 4 1 8 
5 9 3 7 8 1 6 2 4 
6 8 2 3

2 6 7 8 1 5 9 3 4 
1 3 4 7 9 2 8 6 5 
5 9 8 4 3 6 2 7 1 
4 2 3 1 7 8 5 9 6 
8 1 6 5 4 9 7 2 3 
7 5 9 6 2 3 4 1 8 
3 7 5 2 8 1 6 4 9 
9 8 2 3 6 4 1 5 7 
6 4 1 9 5 7 3 8 2 

2 6 7 8 1 5 9 4 3 
1 3 4 7 9 2 8 6 5 
5 9 8 4 3 6 2 7 1 
4 2 3 1 7 8 5 9 6 
8 1 6 5 4 9 7 3 2 
7 5 9 6 2 3 4 1 8 
9 7 5 3 8 1 6 2 4 
3 8 2 9 6 4 1 5 7 
6 4 1 2 5 7 3 8 9 

2 6 7 8 1 5 9 3 4 
1 3 4 7 9 2 8 6 5 
5 9 8 4 3 6 2 7 1 
4 2 3 1 7 8 5 9 6 
8 1 6 5 4 9 7 2 3 
7 5 9 6 2 3 4 1 8 
9 7 5 3 8 1 6 4 2 
3 8 2 9 6 4 1 5 7 
6 4 1 2 5 7 3 8 9 

4 6 7 8 1 2 9 3 5 
1 2 3 7 9 5 8 6 4 
5 9 8 4 3 6 2 7 1 
2 3 4 1 7 8 5 9 6 
8 1 6 5 4 9 7 2 3 
7 5 9 6 2 3 4 1 8 
9 7 5 3 8 1 6 4 2 
3 8 2 9 6 4 1 5 7 
6 4 1 2 5 7 3 8 9 

4 6 7 8 1 2 9 3 5 
1 2 3 7 9 5 8 6 4 
5 9 8 4 3 6 2 7 1 
7 3 4 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 5 9 6 7 3 4 1 8 
9 7 5 3 8 1 6 4 2 
3 8 2 9 6 4 1 5 7 
6 4 1 2 5 7 3 8 9 

4 6 7 8 1 2 9 3 5 
1 2 3 7 9 5 8 6 4 
5 9 8 4 3 6 2 7 1 
7 3 4 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 5 9 6 7 3 4 1 8 
3 7 5 2 8 1 6 4 9 
9 8 2 3

3 6 4 8 1 2 9 7 5 
1 2 5 9 6 7 8 3 4 
9 7 8 4 3 5 2 6 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 5 9 6 7 3 4 1 8 
5 9 3 7 8 1 6 4 2 
6 8 2 3 9 4 1 5 7 
7 4 1 2 5 6 3 8 9 

3 6 5 8 1 2 9 7 4 
1 2 4 9 6 7 8 3 5 
9 7 8 4 3 5 2 6 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 5 9 6 7 3 4 1 8 
5 9 3 7 8 1 6 4 2 
6 8 2 3 9 4 1 5 7 
7 4 1 2 5 6 3 8 9 

5 6 3 8 1 2 9 7 4 
1 2 4 9 6 7 8 3 5 
9 7 8 4 3 5 2 6 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 5 9 6 7 3 4 1 8 
3 9 5 7 8 1 6 4 2 
6 8 2 3 9 4 1 5 7 
7 4 1 2 5 6 3 8 9 

5 6 4 8 1 2 9 7 3 
1 2 3 9 6 7 8 4 5 
9 7 8 4 3 5 2 6 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 3 2 
2 5 9 6 7 3 4 1 8 
3 9 5 7 8 1 6 2 4 
6 8 2 3 9 4 1 5 7 
7 4 1 2 5 6 3 8 9 

5 6 4 8 1 2 9 3 7 
1 2 3 6 9 7 8 4 5 
9 7 8 4 3 5 2 6 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 5 9 7 6 3 4 1 8 
3 9 5 2 8 1 6 7 4 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

5 6 3 8 1 2 9 4 7 
1 2 4 6 9 7 8 3 5 
9 7 8 4 3 5 2 6 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
2 5 9 7 6 3 4 1 8 
3 9 5 2 8 1 6 7 4 
6 8 2 3

8 1 4 5 6 9 7 2 3 
5 6 9 2 7 3 4 1 8 
3 9 5 7 8 1 6 4 2 
6 8 2 3 9 4 1 5 7 
7 4 1 6 5 2 3 8 9 

4 2 3 8 1 6 9 7 5 
1 7 6 9 2 5 8 4 3 
9 5 8 4 3 7 2 6 1 
2 3 7 1 4 8 5 9 6 
8 1 4 5 6 9 7 3 2 
5 6 9 2 7 3 4 1 8 
3 9 5 7 8 1 6 2 4 
6 8 2 3 9 4 1 5 7 
7 4 1 6 5 2 3 8 9 

3 2 4 8 1 6 9 7 5 
1 7 6 9 2 5 8 4 3 
8 5 9 4 3 7 2 6 1 
2 3 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 3 2 
9 6 5 2 7 3 4 1 8 
5 9 3 7 8 1 6 2 4 
6 8 2 3 9 4 1 5 7 
7 4 1 6 5 2 3 8 9 

3 2 4 8 1 6 9 7 5 
1 7 6 9 2 5 8 3 4 
8 5 9 4 3 7 2 6 1 
2 3 7 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
9 6 5 2 7 3 4 1 8 
5 9 3 7 8 1 6 4 2 
6 8 2 3 9 4 1 5 7 
7 4 1 6 5 2 3 8 9 

3 2 4 8 1 6 9 7 5 
1 6 7 9 2 5 8 3 4 
8 5 9 4 3 7 2 6 1 
2 7 3 1 4 8 5 9 6 
4 1 8 5 6 9 7 2 3 
5 9 6 2 7 3 4 1 8 
9 3 5 7 8 1 6 4 2 
6 8 2 3 9 4 1 5 7 
7 4 1 6 5 2 3 8 9 

4 2 7 8 1 6 9 3 5 
1 3 6 9 2 5 8 7 4 
9 5 8 4 3 7 2 6 1 
2 7 3 1 4 8 5 9 6 
8 1 4 5 6 9 7 2 3 
5 6 9 2 7 3 4 1 8 
3 9 5 7 8 1 6 4 2 
6 8 2 3 9 4 1 5 7 
7 4 1 6 5 2 3 8 9 

4 2 7 8 1 6 9 3 5 
1 3 6 9 5 2 8 7 4 
5 9 8 

2 6 3 8 1 7 9 4 5 
1 5 4 6 9 2 8 3 7 
9 7 8 4 3 5 2 6 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 2 3 
5 2 9 7 6 3 4 1 8 
3 9 5 2 8 1 6 7 4 
6 8 2 3 7 4 1 5 9 
7 4 1 9 5 6 3 8 2 

2 6 3 8 1 7 9 4 5 
1 5 4 9 6 2 8 7 3 
9 7 8 4 3 5 2 6 1 
4 3 7 1 2 8 5 9 6 
8 1 6 5 4 9 7 3 2 
5 2 9 6 7 3 4 1 8 
3 9 5 7 8 1 6 2 4 
6 8 2 3 9 4 1 5 7 
7 4 1 2 5 6 3 8 9 

4 2 5 8 1 7 9 6 3 
1 7 3 9 2 6 8 4 5 
9 8 6 4 3 5 2 7 1 
2 3 4 1 7 8 5 9 6 
6 1 8 5 4 9 7 3 2 
5 9 7 2 6 3 4 1 8 
3 5 9 7 8 1 6 2 4 
8 6 2 3 9 4 1 5 7 
7 4 1 6 5 2 3 8 9 

4 2 3 8 1 7 9 6 5 
1 7 5 9 2 6 8 4 3 
9 8 6 4 3 5 2 7 1 
2 3 4 1 7 8 5 9 6 
6 1 8 5 4 9 7 3 2 
5 9 7 2 6 3 4 1 8 
3 5 9 7 8 1 6 2 4 
8 6 2 3 9 4 1 5 7 
7 4 1 6 5 2 3 8 9 

4 2 3 8 1 7 9 6 5 
1 7 5 9 2 6 8 3 4 
9 8 6 4 3 5 2 7 1 
2 3 4 1 7 8 5 9 6 
6 1 8 5 4 9 7 2 3 
5 9 7 2 6 3 4 1 8 
3 5 9 7 8 1 6 4 2 
8 6 2 3 9 4 1 5 7 
7 4 1 6 5 2 3 8 9 

4 2 3 8 1 7 9 6 5 
1 7 5 9 6 2 8 3 4 
9 8 6 4 3 5 2 7 1 
2 3 4 1 7 8 5 9 6 
6 1 8 5 4 9 7 2 3 
5 9 7 6 2 3 4 1 8 
3 5 9 7 8 1 6 4 2 
8 6 2 3

All I had to do make the program generate many different Sudoku solutions was to change the first constraint from a 7 to an 8, making the solver return $\textit{MANY}$ solutions instead of just 1, since that one change made many more solutions possible.

### 16.7.2.12

Your answer goes here.

## TODO Explore a Powerful Tool

Do the following NON-book exercise:

SAT, while being NP-Complete, is a "workhorse of a tool." This exercise asks you to get a taste of running the CryptoMinisat tool on a non-trivial SAT formula. Click on https://msoos.github.io/cryptominisat_web/ to access this tool. When it comes up, it has a prefilled formula. There is a Play button that you can click, whereupon it solves the SAT instance.

This exercise asks you to replace this SAT instance with something bigger: specifically, the Pigeonhole problem (hole6.cnf) from: https://people.sc.fsu.edu/~jburkardt/data/cnf/cnf.html

Just click the above link, get the hole6.cnf file, and plunk the CNF into the buffer.

Hit Play and report on the execution time (you can just look at your phone's clock). If under 2 seconds, say "negligible" for your answer!

How much time would such a problem take through brute-force enumeration of $2^n$ combinations on a computer that takes a microsecond per variable combination (the $n$ is the number of variables used in the Pigeonhole problem)? HINT: Here is how you read the contents of a CNF file:

```
c File:  hole6.cnf <--- these are comment lines - starts wth a "C"
c...
c
p cnf 42 133
-1     -7
-1     -13
...
0 0
<--- CRUCIAL !! Tells you there are 42 variables and 133 clauses
<--- This line says (!x1 + !x7). The "0" is just end-of-a-clause marker!
<--- This line says (!x1 + !x13)
 12     11     10     9      8      7    0 <--- This clause reads
                                          (x12 + x11 + x10 + x9 + x8 + x7)
```

Okay, now you have all the info you need to calculate the time it takes to enumerate $2^n$ combinations!!



Your (two-part) answer goes here.

1. CryptoMinisat runtime:
2. $2^n$ runtime estimation:

## TODO Read and Learn

List six facts that you found interesting about Boolean SAT in these articles:

https://cacm.acm.org/magazines/2009/8/34498-boolean-satisfiability-from-theoretical-hardness-to-practical-success/fulltext

and

https://en.wikipedia.org/wiki/Boolean_satisfiability_problem

Anything that interested you is fine – theoretical or practical. Please write 1-2 sentences per point that interested you.

Your answer goes here.